<span class='note'>*Make me look good.* Click on the cell below and press <kbd>Ctrl</kbd>-<kbd>Enter</kbd>.</span>

In [1]:
from IPython.core.display import HTML
HTML(open('css/custom.css', 'r').read())

<h5 class='prehead'>SA367 &middot; Mathematical Models for Decision Making &middot; Spring 2021 &middot; Uhan</h5>

<h5 class='lesson'>Lesson 4.</h5>

<h1 class='lesson_title'>The Mileage Running Problem</h1>

## The problem

Professor May B. Wright needs to fly from Baltimore (BWI) to Los Angeles (LAX) to attend a conference.
She thinks this would be the perfect opportunity to accumulate some frequent flyer miles on American Airlines (AA), where she already has Platinum status.

Looking into flights on AA, she sees that every itinerary from BWI to LAX costs roughly the same.
She has a full day to spare for travel, so she wants to know: which sequence of AA domestic flights starting at BWI and ending at LAX over the course of one day will allow her to accumulate the most miles?

* Yes, people actually do this. This is known as __mileage running__. 
    - Apparently, this has become harder to do in recent years.
    - [A somewhat recent article from the New York Times](https://www.nytimes.com/2014/09/14/upshot/the-fadeout-of-the-mileage-run.html).
    - [An older article from Wired](https://www.wired.com/2007/07/mileage-runner/).

## Modeling the problem

* Suppose we have a database of every AA domestic flight on a given day.

* In particular, for each flight, we have:
    - the flight number
    - the origin airport
    - the destination airport
    - the departure time at the origin airport
    - the arrival time at the destination airport
    - the distance traveled in miles

* How can we formulate Professor Wright's problem as a shortest path problem?

## pandas (the package, not the animals)

* In the same folder as this notebook, there is a file called `aa_domestic_flights.csv` with the database described above.

* `.csv` stands for __comma-separated values__.

* We can view `.csv` files in Excel - let's see what's in this file. _Cut to Excel..._

* How can we use this data in Python? With __pandas__.

* pandas is a Python package for data analysis. 
    - It's especially useful for cleaning and manipulating datasets.

* pandas does a lot of stuff &mdash; [here is the official documentation for pandas](http://pandas.pydata.org/pandas-docs/stable/index.html).
    
* In this lesson, we'll use pandas in a very basic way to set up the shortest path problem we formulated above.

* To use pandas, we first need to import it, like this:

In [2]:
import pandas as pd

* A pandas __DataFrame__ is just a two-dimensional table, with rows and columns.

* We can use the `read_csv()` function in pandas to read `aa_domestic_flights.csv` into a DataFrame called `df`, like this:

In [3]:
# Read csv file into a DataFrame
# Designate departure and arrival time columns as dates
df = pd.read_csv('aa_domestic_flights.csv', parse_dates=['DEP_TIME', 'ARR_TIME'])

* By default, `read_csv()` assumes the first row of the csv file contains the names of each column.

* The `parse_dates` argument tells `read_csv()` which columns correspond to dates, so that we can perform date-specific calculations on these columns later.

* [Here is the official documentation for `read_csv()`.](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html)

* It's a good idea to take a quick look at the DataFrame `read_csv()` creates, just in case something went wrong.

* To examine the first 5 rows of a DataFrame, we can use the `.head()` method:

In [ ]:
# Print the first 5 rows of df


In [4]:
# Solution
# Print the first 5 rows of df
df.head()

,FLIGHT,ORIGIN,DEST,DEP_TIME,ARR_TIME,DISTANCE
0,1-BOS-JFK,BOS,JFK,2016-09-01 06:00:00,2016-09-01 07:15:00,187.0
1,40-BOS-ORD,BOS,ORD,2016-09-01 19:12:00,2016-09-01 22:02:00,867.0
2,147-BOS-LAX,BOS,LAX,2016-09-01 15:15:00,2016-09-01 21:45:00,2611.0
3,197-BOS-ORD,BOS,ORD,2016-09-01 15:30:00,2016-09-01 18:24:00,867.0
4,198-BOS-JFK,BOS,JFK,2016-09-01 13:10:00,2016-09-01 14:31:00,187.0


- The first column is the __index__ of the DataFrame. The index provides a label for each row of the DataFrame.

- Right now, the index is sort of uninformative. 

- Since each row corresponds to a flight, it would be nice if the index corresponded to the flight names.

- We can do this using the `.set_index()` method:

In [ ]:
# Set the index to the flight names

# Print the first 5 rows of df


In [5]:
# Solution
# Set the index to the flight names
df = df.set_index("FLIGHT")

# Print the first 5 rows of df
df.head()

,ORIGIN,DEST,DEP_TIME,ARR_TIME,DISTANCE
FLIGHT,,,,,
1-BOS-JFK,BOS,JFK,2016-09-01 06:00:00,2016-09-01 07:15:00,187.0
40-BOS-ORD,BOS,ORD,2016-09-01 19:12:00,2016-09-01 22:02:00,867.0
147-BOS-LAX,BOS,LAX,2016-09-01 15:15:00,2016-09-01 21:45:00,2611.0
197-BOS-ORD,BOS,ORD,2016-09-01 15:30:00,2016-09-01 18:24:00,867.0
198-BOS-JFK,BOS,JFK,2016-09-01 13:10:00,2016-09-01 14:31:00,187.0


* A column by itself is called a **Series**.

* You can select the Series `DEST` of the DataFrame `df` like this:

```python
df["DEST"]
```

* So, to print the Series `DEST`, we could write:

In [ ]:
# Print the DEST column


In [6]:
# Solution
# Print the DEST column
print(df["DEST"])

FLIGHT
1-BOS-JFK       JFK
40-BOS-ORD      ORD
147-BOS-LAX     LAX
197-BOS-ORD     ORD
198-BOS-JFK     JFK
               ... 
621-BOI-DFW     DFW
2357-LBB-DFW    DFW
2571-ANC-DFW    DFW
483-GEG-PHX     PHX
490-GEG-PHX     PHX
Name: DEST, Length: 2607, dtype: object


## Making the data easier to use

- We have successfully imported our data into Python! 

- We could set up our shortest path problem using the DataFrame directly, but this would be a bit cumbersome.

- Let's take some additional steps that will make setting up our shortest path problem a bit easier.

- First, let's get a list of the flights. This will be useful, since the nodes in our shortest path problem correspond to the flights.

- In the DataFrame `df` we defined above, the index consists of the flights.

- We can get a list of the index values of `df` with `list(df.index.values)`.

In [ ]:
# List of flights


In [7]:
# Solution
# List of flights
flights = list(df.index.values)

- Let's check our work and inspect `flights`:

In [ ]:
# Print flights


In [ ]:
# Solution
# Print flights
print(f"Flights: {flights}")

* While we're here, let's make sure we have the right number of flights in the variable `flights`:

In [9]:
# Solution
print(f"Number of flights: {len(flights)}")

Number of flights: 2607


- We also want to easily access the origin, destination, departure time, arrival time, and distance for each flight.

- These correspond to the columns in our DataFrame `df`.

- We can convert DataFrame columns to dictionaries as using the `.to_dict()` on the column of interest, like this:

In [ ]:
# Convert columns to dictionaries


In [10]:
# Solution
# Convert columns to dictionaries
origin = df['ORIGIN'].to_dict()
destination = df['DEST'].to_dict()
departure_time = df['DEP_TIME'].to_dict()
arrival_time = df['ARR_TIME'].to_dict()
distance = df['DISTANCE'].to_dict()

- As a result, we can access information about each flight through these dictionaries as follows:

In [ ]:
# Information about flight 1240-BOS-ORD
print(f"Origin: { }")
print(f"Destination: { }")
print(f"Departure time: { }")
print(f"Arrival time: { }")
print(f"Distance: { }")

In [11]:
# Solution
# Information about flight 1240-BOS-ORD
print(f"Origin: {origin['1240-BOS-ORD']}")
print(f"Destination: {destination['1240-BOS-ORD']}")
print(f"Departure time: {departure_time['1240-BOS-ORD']}")
print(f"Arrival time: {arrival_time['1240-BOS-ORD']}")
print(f"Distance: {distance['1240-BOS-ORD']}")

Origin: BOS
Destination: ORD
Departure time: 2016-09-01 12:15:00
Arrival time: 2016-09-01 15:05:00
Distance: 867.0


## Setting up the shortest path problem in networkx

* Now we're ready to set up the shortest path problem we formulated above.

* First, let's import `networkx` and `bellmanford` so we can use them:

In [12]:
import networkx as nx
import bellmanford as bf

### Adding nodes

* Let's build the shortest path graph, starting with an empty directed graph:

In [ ]:
# Create empty NetworkX digraph


In [13]:
# Solution
# Create empty NetworkX digraph
G = nx.DiGraph()

* Next, let's create a "start" and "end" node.

In [ ]:
# Create start and end nodes


In [14]:
# Solution
# Create start and end nodes
G.add_node("start")
G.add_node("end")

* Now, we need to add a node for each flight, or each row of our database.

In [ ]:
# Add a node for each flight


In [15]:
# Solution
# Add a node for each flight
for flight in flights:
    G.add_node(flight)

* The `.number_of_nodes()` method applied to a `networkx` graph &mdash; well, you can guess what it does. Or, you can just try it out:

In [16]:
# Print number of nodes in G
print(G.number_of_nodes())

2609


### Adding edges

* Now we can go over every pair of flight nodes, and check if we need to add an edge between them.
    - Remember the length of these edges is the <span class="rred">negative</span> of the distance of the first flight.

* To add or subtract times, we need to use `pd.to_timedelta()` &mdash; [here is the documentation](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_timedelta.html).
    - For example, to subtract 30 minutes, we would write 
    ```python
    some_time_variable - pd.to_timedelta(30, unit="m")
    ```

* This might seem awkward, but if you think about it, working with dates and time _is_ awkward &mdash; you need to keep track of different (non-base-10) units.

In [ ]:
# Iterate through every pair of flight nodes
        
        # If the first flight arrives where the second flight departs...
            
            # And if the first flight arrives 45 minutes before the second flight leaves,
            # add an edge from the first flight to the second


In [17]:
# Solution
# Iterate through every pair of flight nodes
for first in flights:
    for second in flights:
        
        # If the first flight arrives where the second flight departs...
        if (destination[first] == origin[second]):
            
            # And if the first flight arrives 45 minutes before the second flight leaves,
            # add an edge from the first flight to the second
            if (arrival_time[first] + pd.to_timedelta(45, unit="m") < departure_time[second]):
                G.add_edge(first, second, length=-distance[first])

* Finally, we need to add edges:
    - from the start node to all flights departing from BWI, and
    - from all flights arriving at LAX to the end node.

In [ ]:
# Iterate through all flights

    # If the flight departs from BWI, 
    # add an edge from start to this flight
        
    # If the flight arrives at LAX, 
    # add an edge from this flight to end


In [18]:
# Solution
# Iterate through all flights
for flight in flights:

    # If the flight departs from BWI, 
    # add an edge from start to this flight
    if origin[flight] == "BWI":
        G.add_edge("start", flight, length=0)
        
    # If the flight arrives at LAX, 
    # add an edge from this flight to end
    if destination[flight] == "LAX":
        G.add_edge(flight, "end", length=-distance[flight])

* Similar to `G.number_of_nodes()`, we can perform a sanity check with our work with `G.number_of_edges()`.

In [19]:
# Print the number of edges in G
print(G.number_of_edges())

158335


## Solving the shortest path problem, interpreting the output

* Now that we have our directed graph set up, we can solve for the shortest path from the start node to the end node just like we did in the last lesson:

In [20]:
# Solve the shortest path problem using Bellman-Ford
length, nodes, negative_cycle = bf.bellman_ford(G, source="start", target="end", weight="length")

# Print output from Bellman-Ford
print(f"Negative cycle? {negative_cycle}")
print(f"Shortest path length: {length}")
print(f"Shortest path: {nodes}")

Negative cycle? False
Shortest path length: -8005.0
Shortest path: ['start', '1817-BWI-CLT', '658-CLT-LAS', '1584-LAS-PHX', '694-PHX-HNL', '298-HNL-LAX', 'end']


* What does the output tell us about how to solve Professor Wright's problem?

_Write your notes here. Double-click to edit._

_Solution._

- The length of the shortest path is the negative of the maximum total distance Professor Wright can travel from BWI to LAX. In this case, the maximum total distance is 8005 miles.

- The nodes in the shortest path tells us which flights Professor Wright should take:
    - 1817 from BWI to CLT
    - 658 from CLT to LAS
    - 1584 from LAS to PHX
    - 694 from PHX to HNL
    - 298 from HNL to LAX

## On your own...

Suppose Professor Wright wants to find the longest itinerary from IAD (Washington DC - Dulles) to SAN (San Diego) instead.

In the cell below, write the code that sets up and solves the shortest path formulation for her problem from start to finish.

In the cell after, describe in words what the output from the Bellman-Ford algorithm tells you about how to solve Professor Wright's problem.

In [21]:
# Solution
# Import packages
import pandas as pd
import networkx as nx
import bellmanford as bf

# Read csv file into a DataFrame
# Designate departure and arrival time columns as dates
df = pd.read_csv('aa_domestic_flights.csv', parse_dates=['DEP_TIME', 'ARR_TIME'])

# Create empty networkx digraph
G = nx.DiGraph()

# Create start and end nodes
G.add_node("start")
G.add_node("end")

# Add a node for each flight
for flight in flights:
    G.add_node(flight)
    
# Iterate through every pair of flight nodes
for first in flights:
    for second in flights:
        
        # If the first flight arrives where the second flight departs...
        if (destination[first] == origin[second]):
            
            # And if the first flight arrives 45 minutes before the second flight leaves,
            # add an edge from the first flight to the second
            if (arrival_time[first] + pd.to_timedelta(45, unit="m") < departure_time[second]):
                G.add_edge(first, second, length=-distance[first])

# Iterate through all flights
for flight in flights:

    # If the flight departs from IAD, 
    # add an edge from start to this flight
    if origin[flight] == "IAD":
        G.add_edge("start", flight, length=0)
        
    # If the flight arrives at SAN, 
    # add an edge from this flight to end
    if destination[flight] == "SAN":
        G.add_edge(flight, "end", length=-distance[flight])
        
# Solve the shortest path problem using Bellman-Ford
length, nodes, negative_cycle = bf.bellman_ford(G, source="start", target="end", weight="length")

# Print output from Bellman-Ford
print(f"Negative cycle? {negative_cycle}")
print(f"Shortest path length: {length}")
print(f"Shortest path: {nodes}")

Negative cycle? False
Shortest path length: -6005.0
Shortest path: ['start', '2636-IAD-LAX', '2503-LAX-ORD', '2375-ORD-DFW', '435-DFW-SAN', 'end']


_Write your notes here. Double-click to edit._

_Solution._

- The length of the shortest path is the negative of the maximum total distance Professor Wright can travel from BWI to LAX. In this case, the maximum total distance is 6005 miles.

- The nodes in the shortest path tells us which flights Professor Wright should take:
    - 2636 from IAD to LAX
    - 2503 from LAX to ORD
    - 2375 from ORD to DFW
    - 435 from DFW to SAN